In [2]:
# Import required libraries
import pandas as pd
import json
import requests
import csv

# Download the JSON data from the URL
url = 'https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json'
response = requests.get(url)
data = response.json()

In [1]:
# Read through to understand the structure of the dataset
#data

In [3]:
# Task 1.1
# Extracting the required fields and checking if the returned results was accurate
for result in data:
    for restaurant_data in result['restaurants']:
        restaurant = restaurant_data['restaurant']
        restaurant_id = restaurant['id']
        restaurant_name = restaurant['name']
        restaurant_city = restaurant['location']['city']
        restaurant_country = restaurant['location']['country_id']
        votes = restaurant['user_rating']['votes']
        cuisines = restaurant['cuisines']
        user_rating = restaurant['user_rating']['aggregate_rating']

        #print(f"Restaurant ID : {restaurant_id}\nRestaurant Name: {restaurant_name}\nCuisines: {cuisines}\nCity: {restaurant_city}\nUser Aggregate Rating: {user_rating}\nVotes : {votes}\nCountry ID: {restaurant_country}\n---")

In [4]:
# Task 1.1
# Importing the data from excel regarding the Country Codes

excel_file_path = r'C:\Users\User\Downloads\Country-Code.xlsx'

# Read Excel file into a pandas DataFrame
df = pd.read_excel(excel_file_path)

# Display the DataFrame
print(df)

    Country Code         Country
0              1           India
1             14       Australia
2             30          Brazil
3             37          Canada
4             94       Indonesia
5            148     New Zealand
6            162     Phillipines
7            166           Qatar
8            184       Singapore
9            189    South Africa
10           191       Sri Lanka
11           208          Turkey
12           214             UAE
13           215  United Kingdom
14           216   United States


In [5]:
# Task 1.1
# Initializing a list with country code information to compare against the country id from the restuarant dataset given
country_mapping = {
    1: 'India',
    14: 'Australia',
    30: 'Brazil',
    37: 'Canada',
    94: 'Indonesia',
    148: 'New Zealand',
    162: 'Philippines',
    166: 'Qatar',
    184: 'Singapore',
    189: 'South Africa',
    191: 'Sri Lanka',
    208: 'Turkey',
    214: 'UAE',
    215: 'United Kingdom',
    216: 'United States'
}

In [6]:
# Task 1.1
# Importing relevant library
import csv

# Initializing a empty list to store required fields
updated_restaurants_data = []

# Iterating through dataset 
for result in data:
    for restaurant_data in result['restaurants']:
        restaurant = restaurant_data['restaurant']
        restaurant_id = restaurant['id']
        restaurant_name = restaurant['name']
        restaurant_city = restaurant['location']['city']
        restaurant_country_id = restaurant['location']['country_id']
        votes = restaurant['user_rating']['votes']
        cuisines = restaurant['cuisines']
        user_rating = restaurant['user_rating']['aggregate_rating']

        # Check if the country_id is present in the mapping
        if restaurant_country_id in country_mapping:
            # Update the restaurant_country with the mapped country name
            restaurant_country = country_mapping[restaurant_country_id]

            # Create a dictionary with the desired fields
            restaurant_info = {
                'Restaurant ID': restaurant_id,
                'Restaurant Name': restaurant_name,
                'Country': restaurant_country,
                'City': restaurant_city,
                'User Rating Votes': votes,
                'User Aggregate Rating': user_rating,
                'Cuisines': cuisines,
            }

            # Append the dictionary to the list
            updated_restaurants_data.append(restaurant_info)

# Specify the file path for the CSV file
csv_file_path = 'restaurants.csv'

# Specify the field names for the CSV header
field_names = ['Restaurant ID', 'Restaurant Name', 'Country', 'City', 'User Rating Votes', 'User Aggregate Rating', 'Cuisines']

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=field_names)
    
    # Write the header
    csv_writer.writeheader()
    
    # Write the data
    csv_writer.writerows(updated_restaurants_data)
    
# Double check to see if updated dataset is saved correctly
# print(f'Data has been saved to {csv_file_path}')

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Print all rows to check if results have any significant errors
# pd.set_option('display.max_rows', None)
# df

In [7]:
# Task 1.2 
# Initialize lists to store extracted data
restaurant_info_list = []

# Loop through the data to extract information
for result in data:
    for restaurant_data in result['restaurants']:
        restaurant = restaurant_data['restaurant']
        
        # Extract restaurant information
        restaurant_id = restaurant['R']['res_id']
        restaurant_name = restaurant['name']
        
        # Extract event information if available
        if 'zomato_events' in restaurant:
            for event in restaurant['zomato_events']:
                event_id = event['event']['event_id']
                start_date = event['event']['start_date']
                end_date = event['event']['end_date']
                event_title = event['event']['title']
                
                # Check if the event occurred in April 2019
                if start_date.startswith('2019-04') or end_date.startswith('2019-04'):
                    # Check if the 'photos' list is not empty
                    if event['event']['photos']:
                        # Access the first element of the 'photos' list
                        photo_url = event['event']['photos'][0]['photo']['url']
                    else:
                        photo_url = 'NA'  # Set to None if no photo is available
                    
                    # Append the information to the list
                    restaurant_info_list.append({
                        'Restaurant ID': restaurant_id,
                        'Restaurant Name': restaurant_name,
                        'Event ID': event_id,
                        'Photo URL': photo_url,
                        'Event Start Date': start_date,
                        'Event End Date': end_date,
                        'Event Title': event_title
                    })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(restaurant_info_list)

# Print all rows to ensure that there are no significant errors
# pd.set_option('display.max_rows', None)
# df


In [10]:
# Task 1.2
# Specify the file path for the CSV file
csv_file_path = 'restaurant_events_.csv'

# Save the DataFrame to the CSV file
df.to_csv(csv_file_path, index=False)

# Check if data has been saved correctly
print(f'Data has been saved to {csv_file_path}')

Data has been saved to restaurant_events_.csv


In [11]:
# Task 1.3
# Initialize dictionaries to store counts and sums for each rating text
rating_counts = {'Excellent': 0, 'Very Good': 0, 'Good': 0, 'Average': 0, 'Poor': 0}
rating_sums = {'Excellent': 0, 'Very Good': 0, 'Good': 0, 'Average': 0, 'Poor': 0}

# Loop through the data to calculate counts and sums for each rating text
for result in data:
    for restaurant_data in result['restaurants']:
        restaurant = restaurant_data['restaurant']
        rating_text = restaurant['user_rating']['rating_text']
        aggregate_rating = float(restaurant['user_rating']['aggregate_rating'])
        
        # Check the rating text and update counts and sums accordingly
        if rating_text in rating_counts:
            rating_counts[rating_text] += 1
            rating_sums[rating_text] += aggregate_rating

# Calculate the average rating for each category
rating_thresholds = {}
for rating_text, count in rating_counts.items():
    if count > 0:
        average_rating = rating_sums[rating_text] / count
        rating_thresholds[rating_text] = average_rating

# Display the rating thresholds
print("Rating Thresholds:")
for rating_text, threshold in rating_thresholds.items():
    print(f"{rating_text}: {threshold:.2f}")

Rating Thresholds:
Excellent: 4.67
Very Good: 4.22
Good: 3.78
Average: 3.19
Poor: 2.20
